# Train classifier heads

- This notebook runs multiple experiments with Linear and MLP classification heads, and different modality combinations
- Saves both the best head model and the logs for each combo

In [1]:
import pandas as pd
import geopandas as gpd

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset, TensorDataset, random_split
from torchvision import transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import f1_score
import numpy as np

## For consistent shuffling between experiments

In [2]:
seed = 42
torch.manual_seed(seed)
g = torch.Generator().manual_seed(seed)

## Set up a location embedder

- We use rescaled lat/lon values which are centroids of ONS 1km grid
- We need to define it before torch data loader so we can add locs to our datasets

In [3]:
metadata_cols = ['gridimage_id', 'title', 'grid_reference']
metadata = pd.concat([
    pd.read_csv('kaggle_data/metadata.csv', on_bad_lines='skip', usecols=metadata_cols),
    pd.read_csv('kaggle_data/metadata-extra.csv', on_bad_lines='skip', encoding='latin1', usecols=metadata_cols)
])

In [28]:
grid2coords = (
    gpd.read_file(
        'misc/os_bng_grids.gpkg',
        layer='1km_grid'
    ).rename(
        columns={'tile_name': 'grid_reference'}
    )
    .to_crs(4326)
    .assign(
        geom_=lambda gdf_: gdf_.geometry.centroid,
        lat=lambda gdf_: gdf_.geom_.y,
        lon=lambda gdf_: gdf_.geom_.x
    )
    .set_index('grid_reference')
    [['lat', 'lon']].to_dict(orient='index')
)

img2grid = metadata.set_index('gridimage_id').grid_reference.to_dict()


# UK bounds to rescale
uk = {
    'lat': {'min': 49.9, 'max': 61.9},
    'lon': {'min': -8.6, 'max': 2.1}
}

def get_loc_emb(img_id):
    """Get rescaled lat/lon values"""
    try:
        coords_dict = grid2coords[img2grid[img_id]]
        lat, lon = coords_dict['lat'], coords_dict['lon']

        lat_norm = (lat - uk['lat']['min']) / (uk['lat']['max'] - uk['lat']['min'])
        lon_norm = (lon - uk['lon']['min']) / (uk['lon']['max'] - uk['lon']['min'])
    
        return torch.tensor([lat_norm, lon_norm], dtype=torch.float32)
    except:
        return torch.tensor([0, 0], dtype=torch.float32)

def add_location(batch):
    # img_id, img_f, txt_f, loc_f, y
    # batch is a list of items from full_ds[idx], e.g.
    #   [(img_id0, img_f0, txt_f0, label0), (img_id1, img_f1, txt_f1, label1), …]
    
    img_ids, img_fs, txt_fs, loc_fs, labels = [], [], [], [], []
    
    for img_id, img_f, txt_f, label in batch:
        img_ids.append(img_id)
        img_fs.append(img_f)
        txt_fs.append(txt_f)
        loc_fs.append(get_loc_emb(img_id))
        labels.append(label)

    return (
        torch.stack(img_ids),
        torch.stack(img_fs),
        torch.stack(txt_fs),
        torch.stack(loc_fs),
        torch.stack(labels)
    )

/var/folders/w6/kmxyhb092_3gsgy99f9_t01r0000gn/T/ipykernel_30100/3917409025.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geom_=lambda gdf_: gdf_.geometry.centroid,


## Load datasets

In [5]:
ds = torch.load('splits/dataset.pt', weights_only=False)
train_idx = torch.load('splits/train_idx.pt', weights_only=False)
val_idx = torch.load('splits/val_idx.pt', weights_only=False)

In [6]:
train_dl = DataLoader(
    Subset(ds, train_idx),
    batch_size=4096, shuffle=True,  num_workers=0, generator=g, # consistent shuffling
    collate_fn=add_location # Add location embeddings
)

val_dl = DataLoader(
    Subset(ds, val_idx),
    batch_size=4096, shuffle=False, num_workers=0,
    collate_fn=add_location
)

In [7]:
print("Samples in training:", len(train_idx))
print("Samples in validation:", len(val_idx))

Samples in training: 517679
Samples in validation: 129420


### Define MixUp helper function

In [8]:
def mixup(features, labels, alpha=0.4):
    """Return mixed features and mixed labels."""
    batch_size = features.size(0)
    # sample mixing coefficient
    lam = torch.distributions.Beta(alpha, alpha).sample((batch_size,)).to(features.device)
    lam = torch.max(lam, 1-lam)           # ensure lam >= 0.5
    lam = lam.view(batch_size, 1)         # shape [B,1]
    
    # shuffle batch
    idx = torch.randperm(batch_size)
    f2, y2 = features[idx], labels[idx]
    
    # mix
    mixed_f = lam * features + (1-lam) * f2
    mixed_y = lam * labels   + (1-lam) * y2   # soft labels
    
    return mixed_f, mixed_y

## Define our main experiment function

In [9]:
def run_experiment(head, use_img, use_txt, use_loc, use_mixup):
    """
    Runs a single experiment and saves best head/training log.
    `head` is 'mlp' or 'linear'.
    `use_img`, `use_txt`, `use_loc`, `use_mixup` are all booleans.
    Two files will be generated: the best head in ./heads, and training logs in ./logs.
    """
    
    experiment_name = f'{head}{"-img" if use_img else ""}{"-txt" if use_txt else ""}{"-loc" if use_loc else ""}{"-mixup" if use_mixup else ""}'
    num_labels = 49
    device = 'cpu'
    best_val = 0.0
    best_f1 = 0.0
    best_epoch = -1
    patience = 10 # Stop if no improvement for 10 epochs
    patience_counter = 0
    epochs = 200
    log_lines = []
    loss_fn = nn.BCEWithLogitsLoss()

    # Calculate classifier input dimension
    input_dim = sum([
        512 if use_img else 0,
        512 if use_txt else 0,
        2 if use_loc else 0
    ])

    # Set up classifier head
    if head == 'mlp':
        head = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_labels)
        ).to(device)
        opt = AdamW(head.parameters(), lr=3e-3, weight_decay=1e-4)
    elif head == 'linear':
        head = nn.Linear(input_dim, num_labels).to(device)
        opt = AdamW(head.parameters(), lr=5e-3, weight_decay=1e-5)
    else:
        print("Classification head must be `mlp` or `linear`!")
        return
    
    sch = CosineAnnealingLR(opt, T_max=epochs//4) # Learning schedule
    
    for epoch in range(epochs):
        head.train()
        tot = 0
        
        for img_id, img_f, txt_f, loc_f, y in train_dl:
            opt.zero_grad()
    
            # Combine modalities into a single embedding
            fused_parts = []
            if use_img:
                fused_parts.append(img_f)
            if use_txt:
                fused_parts.append(txt_f)
            if use_loc:
                fused_parts.append(loc_f)
                
            fused = torch.cat(fused_parts, dim=1)

            if use_mixup:
                mixed_f, mixed_y = mixup(fused, y, alpha=0.2)
                logits = head(mixed_f)
                loss = loss_fn(logits, mixed_y)
            else:
                logits = head(fused)
                loss = loss_fn(logits, y.float())
            
            loss.backward()
            opt.step()
            tot += loss.item() * fused.size(0)
            
        sch.step()
    
        # val‐subset accuracy
        head.eval()
        correct, total = 0, 0
        all_preds, all_targets = [], [] # for per-class F-score
        with torch.no_grad():
            for img_id, img_f, txt_f, loc_f, y in val_dl:
                
                # Combine modalities into a single embedding
                fused_parts = []
                if use_img:
                    fused_parts.append(img_f)
                if use_txt:
                    fused_parts.append(txt_f)
                if use_loc:
                    fused_parts.append(loc_f)
                    
                fused = torch.cat(fused_parts, dim=1)
                
                probs = torch.sigmoid(head(fused))
                preds = (probs > 0.5)
    
                # Store for F1 calculation
                all_preds.append(preds.cpu().numpy())
                all_targets.append(y.cpu().numpy())
            
                # Subset accuracy: all 49 match?
                correct += (preds.eq(y.bool()).all(dim=1).sum().item())
                total   += fused.size(0)
    
        # Calculate average (macro) F1 score
        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        macro_f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
        
        train_loss = tot/len(train_idx)
        val_acc = correct/total

        log_line = f"Epoch {epoch:02d} \t train_loss {train_loss:.4f} \t val_acc {val_acc:.4f} \t macro_f1 {macro_f1:.4f}"
        log_lines.append(log_line)
        print(log_line)
    
        if val_acc > best_val:
            best_val = val_acc
            best_f1 = macro_f1
            best_epoch = epoch
            patience_counter = 0
            # Save best head
            torch.save(head.state_dict(), f'heads/{experiment_name}.pth')
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch} (no improvement for {patience} epochs)")
            break
    
    log_line = f"======\nBest epoch {best_epoch:02d} \t val_acc {best_val:.4f} \t macro_f1 {best_f1:.4f}"
    log_lines.append(log_line)
    print(log_line)

    # Save logs
    with open(f'logs/{experiment_name}.txt', 'w') as f:
        f.write('\n'.join(log_lines))

## Run all combinations
### Image embeddings only

In [10]:
%%time
run_experiment('linear', use_img=True, use_txt=False, use_loc=False, use_mixup=False)

Epoch 00 	 train_loss 0.1291 	 val_acc 0.1717 	 macro_f1 0.3018
Epoch 01 	 train_loss 0.0954 	 val_acc 0.1955 	 macro_f1 0.3570
Epoch 02 	 train_loss 0.0927 	 val_acc 0.2021 	 macro_f1 0.3748
Epoch 03 	 train_loss 0.0915 	 val_acc 0.2067 	 macro_f1 0.3836
Epoch 04 	 train_loss 0.0907 	 val_acc 0.2099 	 macro_f1 0.3872
Epoch 05 	 train_loss 0.0902 	 val_acc 0.2111 	 macro_f1 0.3923
Epoch 06 	 train_loss 0.0898 	 val_acc 0.2133 	 macro_f1 0.3945
Epoch 07 	 train_loss 0.0895 	 val_acc 0.2149 	 macro_f1 0.4011
Epoch 08 	 train_loss 0.0893 	 val_acc 0.2159 	 macro_f1 0.3977
Epoch 09 	 train_loss 0.0891 	 val_acc 0.2166 	 macro_f1 0.4030
Epoch 10 	 train_loss 0.0889 	 val_acc 0.2176 	 macro_f1 0.4039
Epoch 11 	 train_loss 0.0888 	 val_acc 0.2162 	 macro_f1 0.4039
Epoch 12 	 train_loss 0.0887 	 val_acc 0.2189 	 macro_f1 0.4054
Epoch 13 	 train_loss 0.0886 	 val_acc 0.2186 	 macro_f1 0.4069
Epoch 14 	 train_loss 0.0885 	 val_acc 0.2190 	 macro_f1 0.4043
Epoch 15 	 train_loss 0.0884 	 val_acc 0

In [11]:
%%time
run_experiment('linear', use_img=True, use_txt=False, use_loc=False, use_mixup=True)

Epoch 00 	 train_loss 0.1354 	 val_acc 0.1767 	 macro_f1 0.3150
Epoch 01 	 train_loss 0.1045 	 val_acc 0.1999 	 macro_f1 0.3730
Epoch 02 	 train_loss 0.1021 	 val_acc 0.2079 	 macro_f1 0.3913
Epoch 03 	 train_loss 0.1011 	 val_acc 0.2116 	 macro_f1 0.4009
Epoch 04 	 train_loss 0.1006 	 val_acc 0.2140 	 macro_f1 0.4051
Epoch 05 	 train_loss 0.1001 	 val_acc 0.2167 	 macro_f1 0.4096
Epoch 06 	 train_loss 0.0999 	 val_acc 0.2177 	 macro_f1 0.4140
Epoch 07 	 train_loss 0.0996 	 val_acc 0.2206 	 macro_f1 0.4168
Epoch 08 	 train_loss 0.0994 	 val_acc 0.2212 	 macro_f1 0.4197
Epoch 09 	 train_loss 0.0993 	 val_acc 0.2208 	 macro_f1 0.4190
Epoch 10 	 train_loss 0.0991 	 val_acc 0.2219 	 macro_f1 0.4214
Epoch 11 	 train_loss 0.0990 	 val_acc 0.2205 	 macro_f1 0.4186
Epoch 12 	 train_loss 0.0989 	 val_acc 0.2230 	 macro_f1 0.4243
Epoch 13 	 train_loss 0.0988 	 val_acc 0.2227 	 macro_f1 0.4235
Epoch 14 	 train_loss 0.0988 	 val_acc 0.2226 	 macro_f1 0.4246
Epoch 15 	 train_loss 0.0987 	 val_acc 0

In [12]:
%%time
run_experiment('mlp', use_img=True, use_txt=False, use_loc=False, use_mixup=False)

Epoch 00 	 train_loss 0.1437 	 val_acc 0.1482 	 macro_f1 0.2292
Epoch 01 	 train_loss 0.1016 	 val_acc 0.1753 	 macro_f1 0.2791
Epoch 02 	 train_loss 0.0984 	 val_acc 0.1856 	 macro_f1 0.3058
Epoch 03 	 train_loss 0.0968 	 val_acc 0.1895 	 macro_f1 0.3152
Epoch 04 	 train_loss 0.0959 	 val_acc 0.1940 	 macro_f1 0.3292
Epoch 05 	 train_loss 0.0951 	 val_acc 0.1992 	 macro_f1 0.3356
Epoch 06 	 train_loss 0.0946 	 val_acc 0.1985 	 macro_f1 0.3380
Epoch 07 	 train_loss 0.0942 	 val_acc 0.2021 	 macro_f1 0.3430
Epoch 08 	 train_loss 0.0938 	 val_acc 0.2042 	 macro_f1 0.3492
Epoch 09 	 train_loss 0.0935 	 val_acc 0.2070 	 macro_f1 0.3532
Epoch 10 	 train_loss 0.0932 	 val_acc 0.2056 	 macro_f1 0.3533
Epoch 11 	 train_loss 0.0930 	 val_acc 0.2056 	 macro_f1 0.3525
Epoch 12 	 train_loss 0.0928 	 val_acc 0.2078 	 macro_f1 0.3531
Epoch 13 	 train_loss 0.0926 	 val_acc 0.2075 	 macro_f1 0.3554
Epoch 14 	 train_loss 0.0925 	 val_acc 0.2099 	 macro_f1 0.3583
Epoch 15 	 train_loss 0.0922 	 val_acc 0

In [13]:
%%time
run_experiment('mlp', use_img=True, use_txt=False, use_loc=False, use_mixup=True)

Epoch 00 	 train_loss 0.1517 	 val_acc 0.1291 	 macro_f1 0.1927
Epoch 01 	 train_loss 0.1115 	 val_acc 0.1655 	 macro_f1 0.2587
Epoch 02 	 train_loss 0.1082 	 val_acc 0.1757 	 macro_f1 0.2823
Epoch 03 	 train_loss 0.1067 	 val_acc 0.1809 	 macro_f1 0.2954
Epoch 04 	 train_loss 0.1057 	 val_acc 0.1868 	 macro_f1 0.3064
Epoch 05 	 train_loss 0.1052 	 val_acc 0.1908 	 macro_f1 0.3164
Epoch 06 	 train_loss 0.1048 	 val_acc 0.1901 	 macro_f1 0.3180
Epoch 07 	 train_loss 0.1043 	 val_acc 0.1941 	 macro_f1 0.3250
Epoch 08 	 train_loss 0.1040 	 val_acc 0.1935 	 macro_f1 0.3237
Epoch 09 	 train_loss 0.1037 	 val_acc 0.1970 	 macro_f1 0.3308
Epoch 10 	 train_loss 0.1034 	 val_acc 0.1968 	 macro_f1 0.3331
Epoch 11 	 train_loss 0.1033 	 val_acc 0.1998 	 macro_f1 0.3383
Epoch 12 	 train_loss 0.1030 	 val_acc 0.2006 	 macro_f1 0.3401
Epoch 13 	 train_loss 0.1029 	 val_acc 0.1993 	 macro_f1 0.3368
Epoch 14 	 train_loss 0.1026 	 val_acc 0.1993 	 macro_f1 0.3386
Epoch 15 	 train_loss 0.1025 	 val_acc 0

### Title embeddings only

In [14]:
%%time
run_experiment('linear', use_img=False, use_txt=True, use_loc=False, use_mixup=False)

Epoch 00 	 train_loss 0.1364 	 val_acc 0.2051 	 macro_f1 0.3694
Epoch 01 	 train_loss 0.0946 	 val_acc 0.2393 	 macro_f1 0.4402
Epoch 02 	 train_loss 0.0911 	 val_acc 0.2517 	 macro_f1 0.4635
Epoch 03 	 train_loss 0.0896 	 val_acc 0.2564 	 macro_f1 0.4733
Epoch 04 	 train_loss 0.0887 	 val_acc 0.2603 	 macro_f1 0.4784
Epoch 05 	 train_loss 0.0881 	 val_acc 0.2639 	 macro_f1 0.4846
Epoch 06 	 train_loss 0.0877 	 val_acc 0.2658 	 macro_f1 0.4892
Epoch 07 	 train_loss 0.0873 	 val_acc 0.2679 	 macro_f1 0.4898
Epoch 08 	 train_loss 0.0871 	 val_acc 0.2681 	 macro_f1 0.4904
Epoch 09 	 train_loss 0.0868 	 val_acc 0.2688 	 macro_f1 0.4932
Epoch 10 	 train_loss 0.0867 	 val_acc 0.2691 	 macro_f1 0.4937
Epoch 11 	 train_loss 0.0865 	 val_acc 0.2709 	 macro_f1 0.4967
Epoch 12 	 train_loss 0.0864 	 val_acc 0.2720 	 macro_f1 0.4983
Epoch 13 	 train_loss 0.0863 	 val_acc 0.2720 	 macro_f1 0.4992
Epoch 14 	 train_loss 0.0862 	 val_acc 0.2726 	 macro_f1 0.4993
Epoch 15 	 train_loss 0.0861 	 val_acc 0

In [15]:
%%time
run_experiment('linear', use_img=False, use_txt=True, use_loc=False, use_mixup=True)

Epoch 00 	 train_loss 0.1444 	 val_acc 0.2052 	 macro_f1 0.3728
Epoch 01 	 train_loss 0.1039 	 val_acc 0.2435 	 macro_f1 0.4516
Epoch 02 	 train_loss 0.1008 	 val_acc 0.2536 	 macro_f1 0.4753
Epoch 03 	 train_loss 0.0994 	 val_acc 0.2610 	 macro_f1 0.4857
Epoch 04 	 train_loss 0.0987 	 val_acc 0.2642 	 macro_f1 0.4922
Epoch 05 	 train_loss 0.0982 	 val_acc 0.2665 	 macro_f1 0.4958
Epoch 06 	 train_loss 0.0978 	 val_acc 0.2689 	 macro_f1 0.5005
Epoch 07 	 train_loss 0.0974 	 val_acc 0.2705 	 macro_f1 0.5023
Epoch 08 	 train_loss 0.0972 	 val_acc 0.2715 	 macro_f1 0.5047
Epoch 09 	 train_loss 0.0970 	 val_acc 0.2720 	 macro_f1 0.5050
Epoch 10 	 train_loss 0.0969 	 val_acc 0.2726 	 macro_f1 0.5082
Epoch 11 	 train_loss 0.0968 	 val_acc 0.2740 	 macro_f1 0.5090
Epoch 12 	 train_loss 0.0967 	 val_acc 0.2744 	 macro_f1 0.5111
Epoch 13 	 train_loss 0.0965 	 val_acc 0.2753 	 macro_f1 0.5118
Epoch 14 	 train_loss 0.0965 	 val_acc 0.2755 	 macro_f1 0.5119
Epoch 15 	 train_loss 0.0964 	 val_acc 0

In [16]:
%%time
run_experiment('mlp', use_img=False, use_txt=True, use_loc=False, use_mixup=False)

Epoch 00 	 train_loss 0.1481 	 val_acc 0.1866 	 macro_f1 0.3106
Epoch 01 	 train_loss 0.0989 	 val_acc 0.2285 	 macro_f1 0.3944
Epoch 02 	 train_loss 0.0952 	 val_acc 0.2428 	 macro_f1 0.4238
Epoch 03 	 train_loss 0.0934 	 val_acc 0.2521 	 macro_f1 0.4391
Epoch 04 	 train_loss 0.0923 	 val_acc 0.2560 	 macro_f1 0.4492
Epoch 05 	 train_loss 0.0914 	 val_acc 0.2611 	 macro_f1 0.4546
Epoch 06 	 train_loss 0.0908 	 val_acc 0.2643 	 macro_f1 0.4586
Epoch 07 	 train_loss 0.0903 	 val_acc 0.2691 	 macro_f1 0.4671
Epoch 08 	 train_loss 0.0899 	 val_acc 0.2683 	 macro_f1 0.4675
Epoch 09 	 train_loss 0.0895 	 val_acc 0.2718 	 macro_f1 0.4729
Epoch 10 	 train_loss 0.0892 	 val_acc 0.2727 	 macro_f1 0.4738
Epoch 11 	 train_loss 0.0890 	 val_acc 0.2738 	 macro_f1 0.4744
Epoch 12 	 train_loss 0.0887 	 val_acc 0.2754 	 macro_f1 0.4783
Epoch 13 	 train_loss 0.0885 	 val_acc 0.2744 	 macro_f1 0.4741
Epoch 14 	 train_loss 0.0884 	 val_acc 0.2776 	 macro_f1 0.4815
Epoch 15 	 train_loss 0.0881 	 val_acc 0

In [17]:
%%time
run_experiment('mlp', use_img=False, use_txt=True, use_loc=False, use_mixup=True)

Epoch 00 	 train_loss 0.1526 	 val_acc 0.1726 	 macro_f1 0.2849
Epoch 01 	 train_loss 0.1084 	 val_acc 0.2174 	 macro_f1 0.3744
Epoch 02 	 train_loss 0.1047 	 val_acc 0.2319 	 macro_f1 0.4040
Epoch 03 	 train_loss 0.1031 	 val_acc 0.2391 	 macro_f1 0.4161
Epoch 04 	 train_loss 0.1021 	 val_acc 0.2473 	 macro_f1 0.4317
Epoch 05 	 train_loss 0.1014 	 val_acc 0.2519 	 macro_f1 0.4366
Epoch 06 	 train_loss 0.1009 	 val_acc 0.2564 	 macro_f1 0.4448
Epoch 07 	 train_loss 0.1005 	 val_acc 0.2580 	 macro_f1 0.4485
Epoch 08 	 train_loss 0.1001 	 val_acc 0.2600 	 macro_f1 0.4508
Epoch 09 	 train_loss 0.0998 	 val_acc 0.2630 	 macro_f1 0.4567
Epoch 10 	 train_loss 0.0996 	 val_acc 0.2624 	 macro_f1 0.4562
Epoch 11 	 train_loss 0.0994 	 val_acc 0.2641 	 macro_f1 0.4575
Epoch 12 	 train_loss 0.0991 	 val_acc 0.2657 	 macro_f1 0.4649
Epoch 13 	 train_loss 0.0990 	 val_acc 0.2655 	 macro_f1 0.4653
Epoch 14 	 train_loss 0.0989 	 val_acc 0.2678 	 macro_f1 0.4653
Epoch 15 	 train_loss 0.0988 	 val_acc 0

## Location only
* Here, validation accuracy never exceeds 0.0 so we don't even save the best model/head

In [29]:
%%time
run_experiment('linear', use_img=False, use_txt=False, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.6381 	 val_acc 0.0000 	 macro_f1 0.0044
Epoch 01 	 train_loss 0.4418 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 02 	 train_loss 0.3290 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 03 	 train_loss 0.2643 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 04 	 train_loss 0.2262 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 05 	 train_loss 0.2027 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 06 	 train_loss 0.1879 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 07 	 train_loss 0.1782 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 08 	 train_loss 0.1718 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 09 	 train_loss 0.1673 	 val_acc 0.0000 	 macro_f1 0.0000
Early stopping at epoch 9 (no improvement for 10 epochs)
Best epoch 09 	 val_acc 0.0000 	 macro_f1 0.0000
CPU times: user 3min 26s, sys: 5min 3s, total: 8min 30s
Wall time: 1min 11s


In [31]:
%%time
run_experiment('linear', use_img=False, use_txt=False, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.6318 	 val_acc 0.0000 	 macro_f1 0.0076
Epoch 01 	 train_loss 0.4380 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 02 	 train_loss 0.3268 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 03 	 train_loss 0.2629 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 04 	 train_loss 0.2252 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 05 	 train_loss 0.2021 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 06 	 train_loss 0.1874 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 07 	 train_loss 0.1779 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 08 	 train_loss 0.1714 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 09 	 train_loss 0.1670 	 val_acc 0.0000 	 macro_f1 0.0000
Early stopping at epoch 9 (no improvement for 10 epochs)
Best epoch 09 	 val_acc 0.0000 	 macro_f1 0.0000
CPU times: user 3min 47s, sys: 4min 49s, total: 8min 36s
Wall time: 1min 7s


In [30]:
%%time
run_experiment('mlp', use_img=False, use_txt=False, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.1885 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 01 	 train_loss 0.1578 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 02 	 train_loss 0.1576 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 03 	 train_loss 0.1575 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 04 	 train_loss 0.1574 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 05 	 train_loss 0.1573 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 06 	 train_loss 0.1572 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 07 	 train_loss 0.1572 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 08 	 train_loss 0.1571 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 09 	 train_loss 0.1571 	 val_acc 0.0000 	 macro_f1 0.0000
Early stopping at epoch 9 (no improvement for 10 epochs)
Best epoch 09 	 val_acc 0.0000 	 macro_f1 0.0000
CPU times: user 4min 2s, sys: 5min 29s, total: 9min 32s
Wall time: 1min 18s


In [32]:
%%time
run_experiment('mlp', use_img=False, use_txt=False, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.1884 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 01 	 train_loss 0.1579 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 02 	 train_loss 0.1577 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 03 	 train_loss 0.1575 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 04 	 train_loss 0.1574 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 05 	 train_loss 0.1573 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 06 	 train_loss 0.1572 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 07 	 train_loss 0.1572 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 08 	 train_loss 0.1572 	 val_acc 0.0000 	 macro_f1 0.0000
Epoch 09 	 train_loss 0.1572 	 val_acc 0.0000 	 macro_f1 0.0000
Early stopping at epoch 9 (no improvement for 10 epochs)
Best epoch 09 	 val_acc 0.0000 	 macro_f1 0.0000
CPU times: user 3min 56s, sys: 5min 33s, total: 9min 29s
Wall time: 1min 17s


### Image + title embeddings

In [18]:
%%time
run_experiment('linear', use_img=True, use_txt=True, use_loc=False, use_mixup=True)

Epoch 00 	 train_loss 0.1192 	 val_acc 0.2681 	 macro_f1 0.4954
Epoch 01 	 train_loss 0.0897 	 val_acc 0.2923 	 macro_f1 0.5411
Epoch 02 	 train_loss 0.0874 	 val_acc 0.3026 	 macro_f1 0.5566
Epoch 03 	 train_loss 0.0864 	 val_acc 0.3082 	 macro_f1 0.5673
Epoch 04 	 train_loss 0.0857 	 val_acc 0.3122 	 macro_f1 0.5740
Epoch 05 	 train_loss 0.0853 	 val_acc 0.3146 	 macro_f1 0.5777
Epoch 06 	 train_loss 0.0850 	 val_acc 0.3171 	 macro_f1 0.5782
Epoch 07 	 train_loss 0.0847 	 val_acc 0.3181 	 macro_f1 0.5821
Epoch 08 	 train_loss 0.0845 	 val_acc 0.3207 	 macro_f1 0.5849
Epoch 09 	 train_loss 0.0843 	 val_acc 0.3209 	 macro_f1 0.5862
Epoch 10 	 train_loss 0.0843 	 val_acc 0.3208 	 macro_f1 0.5881
Epoch 11 	 train_loss 0.0841 	 val_acc 0.3225 	 macro_f1 0.5884
Epoch 12 	 train_loss 0.0840 	 val_acc 0.3235 	 macro_f1 0.5911
Epoch 13 	 train_loss 0.0839 	 val_acc 0.3225 	 macro_f1 0.5895
Epoch 14 	 train_loss 0.0839 	 val_acc 0.3238 	 macro_f1 0.5913
Epoch 15 	 train_loss 0.0838 	 val_acc 0

In [33]:
%%time
run_experiment('linear', use_img=True, use_txt=True, use_loc=False, use_mixup=False)

Epoch 00 	 train_loss 0.1091 	 val_acc 0.2667 	 macro_f1 0.4848
Epoch 01 	 train_loss 0.0778 	 val_acc 0.2912 	 macro_f1 0.5299
Epoch 02 	 train_loss 0.0752 	 val_acc 0.3015 	 macro_f1 0.5442
Epoch 03 	 train_loss 0.0739 	 val_acc 0.3070 	 macro_f1 0.5557
Epoch 04 	 train_loss 0.0731 	 val_acc 0.3121 	 macro_f1 0.5613
Epoch 05 	 train_loss 0.0726 	 val_acc 0.3134 	 macro_f1 0.5642
Epoch 06 	 train_loss 0.0722 	 val_acc 0.3164 	 macro_f1 0.5679
Epoch 07 	 train_loss 0.0719 	 val_acc 0.3173 	 macro_f1 0.5695
Epoch 08 	 train_loss 0.0716 	 val_acc 0.3194 	 macro_f1 0.5733
Epoch 09 	 train_loss 0.0714 	 val_acc 0.3196 	 macro_f1 0.5707
Epoch 10 	 train_loss 0.0713 	 val_acc 0.3203 	 macro_f1 0.5739
Epoch 11 	 train_loss 0.0711 	 val_acc 0.3220 	 macro_f1 0.5772
Epoch 12 	 train_loss 0.0710 	 val_acc 0.3214 	 macro_f1 0.5754
Epoch 13 	 train_loss 0.0709 	 val_acc 0.3222 	 macro_f1 0.5744
Epoch 14 	 train_loss 0.0708 	 val_acc 0.3228 	 macro_f1 0.5785
Epoch 15 	 train_loss 0.0707 	 val_acc 0

In [19]:
%%time
run_experiment('mlp', use_img=True, use_txt=True, use_loc=False, use_mixup=True)

Epoch 00 	 train_loss 0.1411 	 val_acc 0.1881 	 macro_f1 0.2901
Epoch 01 	 train_loss 0.1010 	 val_acc 0.2414 	 macro_f1 0.3976
Epoch 02 	 train_loss 0.0969 	 val_acc 0.2625 	 macro_f1 0.4407
Epoch 03 	 train_loss 0.0950 	 val_acc 0.2743 	 macro_f1 0.4630
Epoch 04 	 train_loss 0.0939 	 val_acc 0.2787 	 macro_f1 0.4699
Epoch 05 	 train_loss 0.0931 	 val_acc 0.2839 	 macro_f1 0.4770
Epoch 06 	 train_loss 0.0925 	 val_acc 0.2866 	 macro_f1 0.4819
Epoch 07 	 train_loss 0.0920 	 val_acc 0.2919 	 macro_f1 0.4926
Epoch 08 	 train_loss 0.0918 	 val_acc 0.2910 	 macro_f1 0.4897
Epoch 09 	 train_loss 0.0915 	 val_acc 0.2939 	 macro_f1 0.4939
Epoch 10 	 train_loss 0.0912 	 val_acc 0.2980 	 macro_f1 0.4992
Epoch 11 	 train_loss 0.0911 	 val_acc 0.2994 	 macro_f1 0.5041
Epoch 12 	 train_loss 0.0908 	 val_acc 0.3002 	 macro_f1 0.5030
Epoch 13 	 train_loss 0.0906 	 val_acc 0.3007 	 macro_f1 0.5048
Epoch 14 	 train_loss 0.0904 	 val_acc 0.3000 	 macro_f1 0.5015
Epoch 15 	 train_loss 0.0902 	 val_acc 0

In [34]:
%%time
run_experiment('mlp', use_img=True, use_txt=True, use_loc=False, use_mixup=False)

Epoch 00 	 train_loss 0.1306 	 val_acc 0.2103 	 macro_f1 0.3338
Epoch 01 	 train_loss 0.0887 	 val_acc 0.2589 	 macro_f1 0.4318
Epoch 02 	 train_loss 0.0845 	 val_acc 0.2790 	 macro_f1 0.4707
Epoch 03 	 train_loss 0.0825 	 val_acc 0.2865 	 macro_f1 0.4859
Epoch 04 	 train_loss 0.0813 	 val_acc 0.2957 	 macro_f1 0.4984
Epoch 05 	 train_loss 0.0804 	 val_acc 0.2989 	 macro_f1 0.5048
Epoch 06 	 train_loss 0.0798 	 val_acc 0.3062 	 macro_f1 0.5164
Epoch 07 	 train_loss 0.0793 	 val_acc 0.3047 	 macro_f1 0.5141
Epoch 08 	 train_loss 0.0790 	 val_acc 0.3095 	 macro_f1 0.5197
Epoch 09 	 train_loss 0.0786 	 val_acc 0.3114 	 macro_f1 0.5235
Epoch 10 	 train_loss 0.0784 	 val_acc 0.3126 	 macro_f1 0.5268
Epoch 11 	 train_loss 0.0781 	 val_acc 0.3152 	 macro_f1 0.5276
Epoch 12 	 train_loss 0.0779 	 val_acc 0.3149 	 macro_f1 0.5277
Epoch 13 	 train_loss 0.0778 	 val_acc 0.3173 	 macro_f1 0.5289
Epoch 14 	 train_loss 0.0776 	 val_acc 0.3151 	 macro_f1 0.5274
Epoch 15 	 train_loss 0.0774 	 val_acc 0

### Image + location

In [20]:
%%time
run_experiment('linear', use_img=True, use_txt=False, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.1354 	 val_acc 0.1750 	 macro_f1 0.3152
Epoch 01 	 train_loss 0.1045 	 val_acc 0.1982 	 macro_f1 0.3716
Epoch 02 	 train_loss 0.1022 	 val_acc 0.2066 	 macro_f1 0.3900
Epoch 03 	 train_loss 0.1012 	 val_acc 0.2120 	 macro_f1 0.4019
Epoch 04 	 train_loss 0.1005 	 val_acc 0.2151 	 macro_f1 0.4084
Epoch 05 	 train_loss 0.1001 	 val_acc 0.2156 	 macro_f1 0.4090
Epoch 06 	 train_loss 0.0998 	 val_acc 0.2178 	 macro_f1 0.4138
Epoch 07 	 train_loss 0.0996 	 val_acc 0.2194 	 macro_f1 0.4151
Epoch 08 	 train_loss 0.0994 	 val_acc 0.2199 	 macro_f1 0.4161
Epoch 09 	 train_loss 0.0992 	 val_acc 0.2204 	 macro_f1 0.4170
Epoch 10 	 train_loss 0.0991 	 val_acc 0.2205 	 macro_f1 0.4207
Epoch 11 	 train_loss 0.0990 	 val_acc 0.2219 	 macro_f1 0.4224
Epoch 12 	 train_loss 0.0989 	 val_acc 0.2225 	 macro_f1 0.4247
Epoch 13 	 train_loss 0.0989 	 val_acc 0.2232 	 macro_f1 0.4224
Epoch 14 	 train_loss 0.0988 	 val_acc 0.2241 	 macro_f1 0.4270
Epoch 15 	 train_loss 0.0987 	 val_acc 0

In [35]:
%%time
run_experiment('linear', use_img=True, use_txt=False, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.1283 	 val_acc 0.1731 	 macro_f1 0.3068
Epoch 01 	 train_loss 0.0953 	 val_acc 0.1950 	 macro_f1 0.3564
Epoch 02 	 train_loss 0.0926 	 val_acc 0.2026 	 macro_f1 0.3739
Epoch 03 	 train_loss 0.0914 	 val_acc 0.2077 	 macro_f1 0.3835
Epoch 04 	 train_loss 0.0907 	 val_acc 0.2093 	 macro_f1 0.3883
Epoch 05 	 train_loss 0.0902 	 val_acc 0.2117 	 macro_f1 0.3944
Epoch 06 	 train_loss 0.0898 	 val_acc 0.2136 	 macro_f1 0.3963
Epoch 07 	 train_loss 0.0895 	 val_acc 0.2150 	 macro_f1 0.3978
Epoch 08 	 train_loss 0.0893 	 val_acc 0.2144 	 macro_f1 0.3971
Epoch 09 	 train_loss 0.0891 	 val_acc 0.2170 	 macro_f1 0.4018
Epoch 10 	 train_loss 0.0889 	 val_acc 0.2190 	 macro_f1 0.4055
Epoch 11 	 train_loss 0.0888 	 val_acc 0.2168 	 macro_f1 0.4018
Epoch 12 	 train_loss 0.0887 	 val_acc 0.2189 	 macro_f1 0.4057
Epoch 13 	 train_loss 0.0886 	 val_acc 0.2178 	 macro_f1 0.4049
Epoch 14 	 train_loss 0.0885 	 val_acc 0.2183 	 macro_f1 0.4065
Epoch 15 	 train_loss 0.0884 	 val_acc 0

In [21]:
%%time
run_experiment('mlp', use_img=True, use_txt=False, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.1509 	 val_acc 0.1353 	 macro_f1 0.2017
Epoch 01 	 train_loss 0.1109 	 val_acc 0.1655 	 macro_f1 0.2599
Epoch 02 	 train_loss 0.1078 	 val_acc 0.1743 	 macro_f1 0.2810
Epoch 03 	 train_loss 0.1065 	 val_acc 0.1829 	 macro_f1 0.2968
Epoch 04 	 train_loss 0.1057 	 val_acc 0.1880 	 macro_f1 0.3094
Epoch 05 	 train_loss 0.1052 	 val_acc 0.1859 	 macro_f1 0.3059
Epoch 06 	 train_loss 0.1047 	 val_acc 0.1904 	 macro_f1 0.3163
Epoch 07 	 train_loss 0.1044 	 val_acc 0.1924 	 macro_f1 0.3230
Epoch 08 	 train_loss 0.1041 	 val_acc 0.1934 	 macro_f1 0.3254
Epoch 09 	 train_loss 0.1038 	 val_acc 0.1954 	 macro_f1 0.3312
Epoch 10 	 train_loss 0.1036 	 val_acc 0.1939 	 macro_f1 0.3274
Epoch 11 	 train_loss 0.1034 	 val_acc 0.1982 	 macro_f1 0.3352
Epoch 12 	 train_loss 0.1032 	 val_acc 0.1953 	 macro_f1 0.3315
Epoch 13 	 train_loss 0.1030 	 val_acc 0.1959 	 macro_f1 0.3317
Epoch 14 	 train_loss 0.1028 	 val_acc 0.1992 	 macro_f1 0.3387
Epoch 15 	 train_loss 0.1026 	 val_acc 0

In [36]:
%%time
run_experiment('mlp', use_img=True, use_txt=False, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.1421 	 val_acc 0.1425 	 macro_f1 0.2184
Epoch 01 	 train_loss 0.1016 	 val_acc 0.1769 	 macro_f1 0.2824
Epoch 02 	 train_loss 0.0982 	 val_acc 0.1846 	 macro_f1 0.3019
Epoch 03 	 train_loss 0.0967 	 val_acc 0.1928 	 macro_f1 0.3181
Epoch 04 	 train_loss 0.0957 	 val_acc 0.1971 	 macro_f1 0.3300
Epoch 05 	 train_loss 0.0950 	 val_acc 0.2004 	 macro_f1 0.3385
Epoch 06 	 train_loss 0.0945 	 val_acc 0.2027 	 macro_f1 0.3437
Epoch 07 	 train_loss 0.0941 	 val_acc 0.2032 	 macro_f1 0.3480
Epoch 08 	 train_loss 0.0937 	 val_acc 0.2055 	 macro_f1 0.3500
Epoch 09 	 train_loss 0.0934 	 val_acc 0.2066 	 macro_f1 0.3528
Epoch 10 	 train_loss 0.0931 	 val_acc 0.2064 	 macro_f1 0.3517
Epoch 11 	 train_loss 0.0929 	 val_acc 0.2086 	 macro_f1 0.3585
Epoch 12 	 train_loss 0.0927 	 val_acc 0.2105 	 macro_f1 0.3585
Epoch 13 	 train_loss 0.0924 	 val_acc 0.2116 	 macro_f1 0.3604
Epoch 14 	 train_loss 0.0923 	 val_acc 0.2110 	 macro_f1 0.3618
Epoch 15 	 train_loss 0.0921 	 val_acc 0

### Title + location

In [22]:
%%time
run_experiment('linear', use_img=False, use_txt=True, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.1438 	 val_acc 0.2052 	 macro_f1 0.3735
Epoch 01 	 train_loss 0.1039 	 val_acc 0.2434 	 macro_f1 0.4531
Epoch 02 	 train_loss 0.1007 	 val_acc 0.2537 	 macro_f1 0.4745
Epoch 03 	 train_loss 0.0994 	 val_acc 0.2607 	 macro_f1 0.4860
Epoch 04 	 train_loss 0.0986 	 val_acc 0.2644 	 macro_f1 0.4930
Epoch 05 	 train_loss 0.0981 	 val_acc 0.2667 	 macro_f1 0.4963
Epoch 06 	 train_loss 0.0978 	 val_acc 0.2688 	 macro_f1 0.5010
Epoch 07 	 train_loss 0.0974 	 val_acc 0.2704 	 macro_f1 0.5025
Epoch 08 	 train_loss 0.0972 	 val_acc 0.2715 	 macro_f1 0.5049
Epoch 09 	 train_loss 0.0970 	 val_acc 0.2718 	 macro_f1 0.5076
Epoch 10 	 train_loss 0.0968 	 val_acc 0.2731 	 macro_f1 0.5085
Epoch 11 	 train_loss 0.0967 	 val_acc 0.2731 	 macro_f1 0.5096
Epoch 12 	 train_loss 0.0966 	 val_acc 0.2734 	 macro_f1 0.5103
Epoch 13 	 train_loss 0.0965 	 val_acc 0.2753 	 macro_f1 0.5113
Epoch 14 	 train_loss 0.0964 	 val_acc 0.2751 	 macro_f1 0.5123
Epoch 15 	 train_loss 0.0965 	 val_acc 0

In [37]:
%%time
run_experiment('linear', use_img=False, use_txt=True, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.1384 	 val_acc 0.2035 	 macro_f1 0.3670
Epoch 01 	 train_loss 0.0948 	 val_acc 0.2386 	 macro_f1 0.4397
Epoch 02 	 train_loss 0.0913 	 val_acc 0.2505 	 macro_f1 0.4611
Epoch 03 	 train_loss 0.0897 	 val_acc 0.2568 	 macro_f1 0.4718
Epoch 04 	 train_loss 0.0888 	 val_acc 0.2606 	 macro_f1 0.4798
Epoch 05 	 train_loss 0.0882 	 val_acc 0.2633 	 macro_f1 0.4832
Epoch 06 	 train_loss 0.0877 	 val_acc 0.2655 	 macro_f1 0.4866
Epoch 07 	 train_loss 0.0874 	 val_acc 0.2666 	 macro_f1 0.4900
Epoch 08 	 train_loss 0.0871 	 val_acc 0.2686 	 macro_f1 0.4939
Epoch 09 	 train_loss 0.0869 	 val_acc 0.2691 	 macro_f1 0.4915
Epoch 10 	 train_loss 0.0867 	 val_acc 0.2698 	 macro_f1 0.4944
Epoch 11 	 train_loss 0.0866 	 val_acc 0.2711 	 macro_f1 0.4971
Epoch 12 	 train_loss 0.0864 	 val_acc 0.2705 	 macro_f1 0.4958
Epoch 13 	 train_loss 0.0863 	 val_acc 0.2715 	 macro_f1 0.4983
Epoch 14 	 train_loss 0.0862 	 val_acc 0.2719 	 macro_f1 0.4986
Epoch 15 	 train_loss 0.0861 	 val_acc 0

In [23]:
%%time
run_experiment('mlp', use_img=False, use_txt=True, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.1531 	 val_acc 0.1714 	 macro_f1 0.2830
Epoch 01 	 train_loss 0.1084 	 val_acc 0.2212 	 macro_f1 0.3805
Epoch 02 	 train_loss 0.1046 	 val_acc 0.2338 	 macro_f1 0.4068
Epoch 03 	 train_loss 0.1031 	 val_acc 0.2424 	 macro_f1 0.4234
Epoch 04 	 train_loss 0.1022 	 val_acc 0.2467 	 macro_f1 0.4309
Epoch 05 	 train_loss 0.1014 	 val_acc 0.2513 	 macro_f1 0.4377
Epoch 06 	 train_loss 0.1009 	 val_acc 0.2538 	 macro_f1 0.4439
Epoch 07 	 train_loss 0.1005 	 val_acc 0.2585 	 macro_f1 0.4497
Epoch 08 	 train_loss 0.1001 	 val_acc 0.2613 	 macro_f1 0.4544
Epoch 09 	 train_loss 0.0998 	 val_acc 0.2613 	 macro_f1 0.4564
Epoch 10 	 train_loss 0.0995 	 val_acc 0.2645 	 macro_f1 0.4605
Epoch 11 	 train_loss 0.0993 	 val_acc 0.2641 	 macro_f1 0.4602
Epoch 12 	 train_loss 0.0990 	 val_acc 0.2659 	 macro_f1 0.4601
Epoch 13 	 train_loss 0.0990 	 val_acc 0.2680 	 macro_f1 0.4656
Epoch 14 	 train_loss 0.0988 	 val_acc 0.2714 	 macro_f1 0.4723
Epoch 15 	 train_loss 0.0986 	 val_acc 0

In [38]:
%%time
run_experiment('mlp', use_img=False, use_txt=True, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.1460 	 val_acc 0.1855 	 macro_f1 0.3081
Epoch 01 	 train_loss 0.0989 	 val_acc 0.2288 	 macro_f1 0.3951
Epoch 02 	 train_loss 0.0950 	 val_acc 0.2425 	 macro_f1 0.4217
Epoch 03 	 train_loss 0.0933 	 val_acc 0.2511 	 macro_f1 0.4381
Epoch 04 	 train_loss 0.0922 	 val_acc 0.2567 	 macro_f1 0.4461
Epoch 05 	 train_loss 0.0914 	 val_acc 0.2608 	 macro_f1 0.4556
Epoch 06 	 train_loss 0.0907 	 val_acc 0.2651 	 macro_f1 0.4626
Epoch 07 	 train_loss 0.0903 	 val_acc 0.2684 	 macro_f1 0.4653
Epoch 08 	 train_loss 0.0899 	 val_acc 0.2677 	 macro_f1 0.4663
Epoch 09 	 train_loss 0.0895 	 val_acc 0.2711 	 macro_f1 0.4707
Epoch 10 	 train_loss 0.0892 	 val_acc 0.2732 	 macro_f1 0.4735
Epoch 11 	 train_loss 0.0890 	 val_acc 0.2731 	 macro_f1 0.4734
Epoch 12 	 train_loss 0.0887 	 val_acc 0.2740 	 macro_f1 0.4759
Epoch 13 	 train_loss 0.0884 	 val_acc 0.2756 	 macro_f1 0.4799
Epoch 14 	 train_loss 0.0882 	 val_acc 0.2771 	 macro_f1 0.4792
Epoch 15 	 train_loss 0.0880 	 val_acc 0

### Image + title + location

In [24]:
%%time
run_experiment('linear', use_img=True, use_txt=True, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.1194 	 val_acc 0.2688 	 macro_f1 0.4964
Epoch 01 	 train_loss 0.0897 	 val_acc 0.2941 	 macro_f1 0.5418
Epoch 02 	 train_loss 0.0874 	 val_acc 0.3039 	 macro_f1 0.5561
Epoch 03 	 train_loss 0.0864 	 val_acc 0.3092 	 macro_f1 0.5689
Epoch 04 	 train_loss 0.0856 	 val_acc 0.3115 	 macro_f1 0.5737
Epoch 05 	 train_loss 0.0853 	 val_acc 0.3145 	 macro_f1 0.5790
Epoch 06 	 train_loss 0.0850 	 val_acc 0.3175 	 macro_f1 0.5811
Epoch 07 	 train_loss 0.0847 	 val_acc 0.3184 	 macro_f1 0.5828
Epoch 08 	 train_loss 0.0845 	 val_acc 0.3208 	 macro_f1 0.5861
Epoch 09 	 train_loss 0.0843 	 val_acc 0.3208 	 macro_f1 0.5850
Epoch 10 	 train_loss 0.0842 	 val_acc 0.3213 	 macro_f1 0.5878
Epoch 11 	 train_loss 0.0840 	 val_acc 0.3223 	 macro_f1 0.5887
Epoch 12 	 train_loss 0.0840 	 val_acc 0.3228 	 macro_f1 0.5912
Epoch 13 	 train_loss 0.0839 	 val_acc 0.3233 	 macro_f1 0.5912
Epoch 14 	 train_loss 0.0839 	 val_acc 0.3246 	 macro_f1 0.5919
Epoch 15 	 train_loss 0.0838 	 val_acc 0

In [39]:
%%time
run_experiment('linear', use_img=True, use_txt=True, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.1088 	 val_acc 0.2659 	 macro_f1 0.4828
Epoch 01 	 train_loss 0.0778 	 val_acc 0.2921 	 macro_f1 0.5296
Epoch 02 	 train_loss 0.0752 	 val_acc 0.3018 	 macro_f1 0.5455
Epoch 03 	 train_loss 0.0739 	 val_acc 0.3078 	 macro_f1 0.5544
Epoch 04 	 train_loss 0.0731 	 val_acc 0.3108 	 macro_f1 0.5600
Epoch 05 	 train_loss 0.0726 	 val_acc 0.3135 	 macro_f1 0.5650
Epoch 06 	 train_loss 0.0722 	 val_acc 0.3161 	 macro_f1 0.5693
Epoch 07 	 train_loss 0.0719 	 val_acc 0.3183 	 macro_f1 0.5685
Epoch 08 	 train_loss 0.0716 	 val_acc 0.3196 	 macro_f1 0.5703
Epoch 09 	 train_loss 0.0714 	 val_acc 0.3204 	 macro_f1 0.5728
Epoch 10 	 train_loss 0.0712 	 val_acc 0.3206 	 macro_f1 0.5746
Epoch 11 	 train_loss 0.0711 	 val_acc 0.3218 	 macro_f1 0.5784
Epoch 12 	 train_loss 0.0710 	 val_acc 0.3220 	 macro_f1 0.5784
Epoch 13 	 train_loss 0.0709 	 val_acc 0.3231 	 macro_f1 0.5765
Epoch 14 	 train_loss 0.0708 	 val_acc 0.3235 	 macro_f1 0.5794
Epoch 15 	 train_loss 0.0707 	 val_acc 0

In [25]:
%%time
run_experiment('mlp', use_img=True, use_txt=True, use_loc=True, use_mixup=True)

Epoch 00 	 train_loss 0.1408 	 val_acc 0.1920 	 macro_f1 0.2995
Epoch 01 	 train_loss 0.1006 	 val_acc 0.2449 	 macro_f1 0.4040
Epoch 02 	 train_loss 0.0967 	 val_acc 0.2616 	 macro_f1 0.4380
Epoch 03 	 train_loss 0.0949 	 val_acc 0.2721 	 macro_f1 0.4578
Epoch 04 	 train_loss 0.0939 	 val_acc 0.2787 	 macro_f1 0.4693
Epoch 05 	 train_loss 0.0931 	 val_acc 0.2822 	 macro_f1 0.4771
Epoch 06 	 train_loss 0.0926 	 val_acc 0.2867 	 macro_f1 0.4825
Epoch 07 	 train_loss 0.0921 	 val_acc 0.2909 	 macro_f1 0.4893
Epoch 08 	 train_loss 0.0917 	 val_acc 0.2938 	 macro_f1 0.4917
Epoch 09 	 train_loss 0.0914 	 val_acc 0.2943 	 macro_f1 0.4926
Epoch 10 	 train_loss 0.0911 	 val_acc 0.2987 	 macro_f1 0.5011
Epoch 11 	 train_loss 0.0909 	 val_acc 0.2980 	 macro_f1 0.4970
Epoch 12 	 train_loss 0.0906 	 val_acc 0.3013 	 macro_f1 0.5031
Epoch 13 	 train_loss 0.0904 	 val_acc 0.3014 	 macro_f1 0.5041
Epoch 14 	 train_loss 0.0903 	 val_acc 0.3007 	 macro_f1 0.5052
Epoch 15 	 train_loss 0.0902 	 val_acc 0

In [40]:
%%time
run_experiment('mlp', use_img=True, use_txt=True, use_loc=True, use_mixup=False)

Epoch 00 	 train_loss 0.1326 	 val_acc 0.2072 	 macro_f1 0.3263
Epoch 01 	 train_loss 0.0891 	 val_acc 0.2557 	 macro_f1 0.4282
Epoch 02 	 train_loss 0.0849 	 val_acc 0.2731 	 macro_f1 0.4631
Epoch 03 	 train_loss 0.0829 	 val_acc 0.2850 	 macro_f1 0.4812
Epoch 04 	 train_loss 0.0816 	 val_acc 0.2937 	 macro_f1 0.4955
Epoch 05 	 train_loss 0.0807 	 val_acc 0.3013 	 macro_f1 0.5072
Epoch 06 	 train_loss 0.0800 	 val_acc 0.3042 	 macro_f1 0.5111
Epoch 07 	 train_loss 0.0795 	 val_acc 0.3081 	 macro_f1 0.5170
Epoch 08 	 train_loss 0.0790 	 val_acc 0.3067 	 macro_f1 0.5177
Epoch 09 	 train_loss 0.0786 	 val_acc 0.3088 	 macro_f1 0.5161
Epoch 10 	 train_loss 0.0783 	 val_acc 0.3136 	 macro_f1 0.5263
Epoch 11 	 train_loss 0.0781 	 val_acc 0.3120 	 macro_f1 0.5229
Epoch 12 	 train_loss 0.0778 	 val_acc 0.3147 	 macro_f1 0.5278
Epoch 13 	 train_loss 0.0776 	 val_acc 0.3168 	 macro_f1 0.5291
Epoch 14 	 train_loss 0.0774 	 val_acc 0.3176 	 macro_f1 0.5343
Epoch 15 	 train_loss 0.0773 	 val_acc 0